In [2]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

In [3]:
def normalize_col(col: str) -> str:
    """
    Normalize column names to snake_case and carry unit markers into names.
    e.g., 'Thickness (mm)' -> 'thickness_mm', 'Gross weight (kg)' -> 'weight_kg'
    """
    c = col.strip()
    c = re.sub(r"\s*\(mm\)\s*", "_mm", c, flags=re.I)
    c = re.sub(r"\s*\(kg\)\s*", "_kg", c, flags=re.I)
    c = re.sub(r"[^A-Za-z0-9]+", "_", c).strip("_").lower()
    return c

In [4]:
def coerce_number(x):
    """
    Extract and parse the first number in a messy string.
    Handles German decimal comma and thousands separators.
    Returns float or NaN.
    """
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)):
        return float(x)
    s = str(x).replace("\xa0", " ")  # non-breaking spaces
    # match numbers like 1.250, 1,50, 1250, -12, +3.5
    m = re.search(r"[-+]?\d{1,3}(?:[.\s]\d{3})*(?:[.,]\d+)?|[-+]?\d+(?:[.,]\d+)?", s)
    if not m:
        return np.nan
    num = m.group(0)
    # remove thousands, normalize decimal
    num = num.replace(" ", "").replace(".", "").replace(",", ".")
    try:
        return float(num)
    except Exception:
        return np.nan

In [5]:
def standardize_finish(value, description=None):
    """
    Map various finish/oiling/pickling/galvanizing terms to a small controlled vocabulary.
    """
    text = " ".join([str(value or ""), str(description or "")]).lower()
    if any(k in text for k in ["gebeizt"]):
        return "pickled"
    if any(k in text for k in ["ungebeizt"]):
        return "unpickled"
    if any(k in text for k in ["geölt", "oiled", "öl", "oel", "oelhaltig"]):
        return "oiled"
    if any(k in text for k in ["verzinkt", "zinc", "+z", "galvanized"]):
        return "galvanized"
    if any(k in text for k in ["blank", "bright"]):
        return "bright"
    if any(k in text for k in ["brushed"]):
        return "brushed"
    return np.nan

In [6]:
def normalize_quality(value):
    """
    Normalize quality/choice labels to '1st'/'2nd'/'3rd'/'4th' where clear.
    Falls back to the original token if ambiguous.
    """
    if pd.isna(value):
        return np.nan
    s = str(value).strip().lower()
    # a few German cues
    s = s.replace("erst", "1st").replace("zweit", "2nd").replace("dritt", "3rd")
    if re.fullmatch(r"(1|1st|i|a|prime)", s):
        return "1st"
    if re.fullmatch(r"(2|2nd|ii|b)", s):
        return "2nd"
    if re.fullmatch(r"(3|3rd|iii|c)", s):
        return "3rd"
    if re.fullmatch(r"(4|4th|iv|d)", s):
        return "4th"
    m = re.match(r"(\d)(?:st|nd|rd|th)?", s)
    if m:
        n = m.group(1)
        suffix = "st" if n == "1" else "nd" if n == "2" else "rd" if n == "3" else "th"
        return f"{n}{suffix}"
    return s

In [7]:
def clean_bool_reserved(x):
    """
    Normalize reserved-like booleans/flags.
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if s in {"y", "yes", "true", "ja", "reserved", "1", "res", "r"}:
        return True
    if s in {"n", "no", "false", "nein", "0", "frei", "available"}:
        return False
    return np.nan

In [8]:
def pick_first_nonnull(*args):
    for a in args:
        if pd.notna(a) and a != "":
            return a
    return np.nan

In [9]:
def parse_dims_from_material(s):
    """
    If Supplier 2 material contains a 't x w' pattern (e.g. '1,50x1250'),
    extract thickness_mm and width_mm as floats.
    """
    if pd.isna(s):
        return pd.Series([np.nan, np.nan])
    txt = str(s).lower().replace("×", "x")
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*x\s*(\d{2,5}(?:[.,]\d+)?)", txt)
    if not m:
        return pd.Series([np.nan, np.nan])
    t_raw, w_raw = m.group(1), m.group(2)
    t = t_raw.replace(".", "").replace(",", ".") if "," in t_raw else t_raw.replace(",", ".")
    w = w_raw.replace(".", "").replace(",", ".") if "," in w_raw else w_raw.replace(",", ".")
    try:
        tval = float(t)
    except Exception:
        tval = np.nan
    try:
        wval = float(w)
    except Exception:
        wval = np.nan
    return pd.Series([tval, wval])

In [10]:
def extract_grade(text):
    """
    Extract common steel grade tokens from a free-text string.
    Examples: DX51D, S235JR, C100S, DC01, HC300, DD11, etc.
    """
    if pd.isna(text):
        return np.nan
    s = str(text).upper()
    m = re.search(r"\b(S\d{3}[A-Z]{0,2}\b|DX\d{2}D\b|DC\d{2}\b|C\d{2,3}S\b|DC0\d\b|HC\d{2,3}\b|DD\d{2}\b)\b", s)
    return m.group(0) if m else np.nan


In [11]:
p1 = Path("/content/supplier_data1.xlsx")
p2 = Path("/content/supplier_data2.xlsx")
df1 = pd.read_excel(p1)
df2 = pd.read_excel(p2)

In [12]:
df1.head()

,Quality/Choice,Grade,Finish,Thickness (mm),Width (mm),Description,Gross weight (kg),RP02,RM,Quantity,AG,AI
0,3rd,C200S,gebeizt und geglüht,2.77,1100,Längs- oder Querisse,13983,333.6,606.2,0.00,16.11,0.0054
1,3rd,C300S,ungebeizt,2.65,1075,Längs- oder Querisse,13047,717.7,0.0,0.00,16.11,0.0046
2,3rd,C100S,gebeizt und geglüht,2.20,1100,Kantenfehler - FS-Kantenrisse,14155,368.9,0.0,10.84,0.00,0.0061
3,2nd,C100S,gebeizt,2.86,1100,Längs- oder Querisse,11381,368.9,601.7,22.87,0.00,0.0062
4,1st,C300S,ungebeizt,2.88,1050,Sollmasse (Gewicht) unterschritten,10072,0.0,1213.0,22.87,0.00,0.0041


In [13]:
df2.head()

,Material,Description,Article ID,Weight (kg),Quantity,Reserved
0,HDC,Material is Oiled,23048203,24469,52,NOT RESERVED
1,S235JR,Material is Oiled,23040547,16984,41,NOT RESERVED
2,S235JR,Material is Painted,23046057,9162,28,NOT RESERVED
3,DX51D +AZ150,Material is Oiled,23041966,12119,66,VANILLA
4,HDC,Material is Painted,23043884,17260,26,NOT RESERVED


In [14]:
df1.isnull().sum()

,0
Quality/Choice,0
Grade,0
Finish,0
Thickness (mm),0
Width (mm),0
Description,0
Gross weight (kg),0
RP02,0
RM,0
Quantity,0


In [15]:
df2.isnull().sum()

,0
Material,0
Description,0
Article ID,0
Weight (kg),0
Quantity,0
Reserved,0


In [16]:
df1.dtypes

,0
Quality/Choice,object
Grade,object
Finish,object
Thickness (mm),float64
Width (mm),int64
Description,object
Gross weight (kg),int64
RP02,float64
RM,float64
Quantity,float64


In [17]:
df2.dtypes

,0
Material,object
Description,object
Article ID,int64
Weight (kg),int64
Quantity,int64
Reserved,object


In [19]:
df1.describe()

,Thickness (mm),Width (mm),Gross weight (kg),RP02,RM,Quantity,AG,AI
count,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000
mean,2.508800,1075.000000,10227.840000,336.39200,653.682000,13.536200,9.706120,0.005500
std,0.273112,48.968961,2900.401516,245.12073,403.594141,10.380396,7.080155,0.000886
min,2.010000,1000.000000,5011.000000,0.00000,0.000000,0.000000,0.000000,0.004100
25%,2.292500,1050.000000,8065.500000,83.40000,601.700000,0.000000,0.000000,0.004725
50%,2.510000,1075.000000,10167.000000,368.90000,606.200000,10.840000,15.050000,0.005450
75%,2.750000,1100.000000,12844.000000,368.90000,1061.300000,22.870000,16.110000,0.006200
max,2.980000,1150.000000,14839.000000,717.70000,1213.000000,25.460000,16.110000,0.007000


In [20]:
# Normalize headers
df1.columns = [normalize_col(c) for c in df1.columns]
df2.columns = [normalize_col(c) for c in df2.columns]

In [21]:
df1.head()

,quality_choice,grade,finish,thickness_mm,width_mm,description,gross_weight_kg,rp02,rm,quantity,ag,ai
0,3rd,C200S,gebeizt und geglüht,2.77,1100,Längs- oder Querisse,13983,333.6,606.2,0.00,16.11,0.0054
1,3rd,C300S,ungebeizt,2.65,1075,Längs- oder Querisse,13047,717.7,0.0,0.00,16.11,0.0046
2,3rd,C100S,gebeizt und geglüht,2.20,1100,Kantenfehler - FS-Kantenrisse,14155,368.9,0.0,10.84,0.00,0.0061
3,2nd,C100S,gebeizt,2.86,1100,Längs- oder Querisse,11381,368.9,601.7,22.87,0.00,0.0062
4,1st,C300S,ungebeizt,2.88,1050,Sollmasse (Gewicht) unterschritten,10072,0.0,1213.0,22.87,0.00,0.0041


In [22]:
df2.head()

,material,description,article_id,weight_kg,quantity,reserved
0,HDC,Material is Oiled,23048203,24469,52,NOT RESERVED
1,S235JR,Material is Oiled,23040547,16984,41,NOT RESERVED
2,S235JR,Material is Painted,23046057,9162,28,NOT RESERVED
3,DX51D +AZ150,Material is Oiled,23041966,12119,66,VANILLA
4,HDC,Material is Painted,23043884,17260,26,NOT RESERVED


In [23]:
# Canonical schema
canonical_cols = [
    "source",
    "article_id",
    "material",
    "grade",
    "quality_choice",
    "finish",
    "thickness_mm",
    "width_mm",
    "weight_kg",
    "quantity",
    "rp02",
    "rm",
    "ag",
    "ai",
    "reserved",
    "description",
]

In [24]:
# ------------------------
# Supplier 1 cleaning
# ------------------------
s1 = df1.copy()
rename_map1 = {
    "quality_choice": "quality_choice",
    "quality": "quality_choice",
    "choice": "quality_choice",
    "grade": "grade",
    "finish": "finish",
    "thickness_mm": "thickness_mm",
    "thickness": "thickness_mm",
    "width_mm": "width_mm",
    "width": "width_mm",
    "gross_weight_kg": "weight_kg",
    "gross_weight": "weight_kg",
    "gewicht_kg": "weight_kg",
    "quantity": "quantity",
    "qty": "quantity",
    "rp02": "rp02",
    "rm": "rm",
    "ag": "ag",
    "ai": "ai",
    "description": "description",
}

In [25]:
s1 = s1.rename(columns={k: v for k, v in rename_map1.items() if k in s1.columns and v != k})

for col in ["thickness_mm", "width_mm", "weight_kg", "quantity", "rp02", "rm", "ag", "ai"]:
    if col in s1.columns:
        s1[col] = s1[col].apply(coerce_number)

In [26]:
# Standardize finish and quality
s1["finish"] = s1.apply(lambda r: standardize_finish(r.get("finish", np.nan),
                                                     r.get("description", np.nan)), axis=1)
s1["quality_choice"] = (
    s1["quality_choice"].apply(normalize_quality) if "quality_choice" in s1.columns else np.nan
)


In [27]:
# Ensure all canonical columns exist
for c in canonical_cols:
    if c not in s1.columns:
        s1[c] = np.nan

s1["source"] = "supplier1"
s1_clean = s1[canonical_cols].copy()

In [28]:
s1_clean.head()

,source,article_id,material,grade,quality_choice,finish,thickness_mm,width_mm,weight_kg,quantity,rp02,rm,ag,ai,reserved,description
0,supplier1,NaN,NaN,C200S,3rd,pickled,2.77,1100.0,13983.0,0.00,333.6,606.2,16.11,0.0054,NaN,Längs- oder Querisse
1,supplier1,NaN,NaN,C300S,3rd,pickled,2.65,1075.0,13047.0,0.00,717.7,0.0,16.11,0.0046,NaN,Längs- oder Querisse
2,supplier1,NaN,NaN,C100S,3rd,pickled,2.20,1100.0,14155.0,10.84,368.9,0.0,0.00,0.0061,NaN,Kantenfehler - FS-Kantenrisse
3,supplier1,NaN,NaN,C100S,2nd,pickled,2.86,1100.0,11381.0,22.87,368.9,601.7,0.00,0.0062,NaN,Längs- oder Querisse
4,supplier1,NaN,NaN,C300S,1st,pickled,2.88,1050.0,10072.0,22.87,0.0,1213.0,0.00,0.0041,NaN,Sollmasse (Gewicht) unterschritten


In [29]:
# ------------------------
# Supplier 2 cleaning
# ------------------------
s2 = df2.copy()
rename_map2 = {
    "material": "material",
    "description": "description",
    "article_id": "article_id",
    "weight_kg": "weight_kg",
    "weight": "weight_kg",
    "quantity": "quantity",
    "qty": "quantity",
    "reserved": "reserved",
    "thickness_mm": "thickness_mm",
    "thickness": "thickness_mm",
    "width_mm": "width_mm",
    "width": "width_mm",
}

In [30]:
s2 = s2.rename(columns={k: v for k, v in rename_map2.items() if k in s2.columns and v != k})

# If dims are embedded in the 'material' string, parse them
if "thickness_mm" not in s2.columns or "width_mm" not in s2.columns:
    s2[["thickness_mm", "width_mm"]] = s2["material"].apply(parse_dims_from_material)


In [31]:
# Numerics
for col in ["thickness_mm", "width_mm", "weight_kg", "quantity"]:
    if col in s2.columns:
        s2[col] = s2[col].apply(coerce_number)

In [32]:
# Reserved flag
s2["reserved"] = s2["reserved"].apply(clean_bool_reserved) if "reserved" in s2.columns else np.nan

# Finish from description/material
s2["finish"] = s2.apply(
    lambda r: standardize_finish(None, pick_first_nonnull(r.get("description", np.nan),
                                                          r.get("material", np.nan))),
    axis=1
)


In [33]:
# Grade extraction
s2["grade"] = s2["material"].apply(extract_grade) if "material" in s2.columns else np.nan

# Ensure all canonical columns exist
for c in canonical_cols:
    if c not in s2.columns:
        s2[c] = np.nan

s2["source"] = "supplier2"
s2_clean = s2[canonical_cols].copy()

In [34]:
s2_clean.head()

,source,article_id,material,grade,quality_choice,finish,thickness_mm,width_mm,weight_kg,quantity,rp02,rm,ag,ai,reserved,description
0,supplier2,23048203,HDC,NaN,NaN,oiled,NaN,NaN,24469.0,52.0,NaN,NaN,NaN,NaN,NaN,Material is Oiled
1,supplier2,23040547,S235JR,S235JR,NaN,oiled,NaN,NaN,16984.0,41.0,NaN,NaN,NaN,NaN,NaN,Material is Oiled
2,supplier2,23046057,S235JR,S235JR,NaN,NaN,NaN,NaN,9162.0,28.0,NaN,NaN,NaN,NaN,NaN,Material is Painted
3,supplier2,23041966,DX51D +AZ150,DX51D,NaN,oiled,NaN,NaN,12119.0,66.0,NaN,NaN,NaN,NaN,NaN,Material is Oiled
4,supplier2,23043884,HDC,NaN,NaN,NaN,NaN,NaN,17260.0,26.0,NaN,NaN,NaN,NaN,NaN,Material is Painted


In [35]:
# ------------------------
# Merge & finalize
# ------------------------
inventory = pd.concat([s1_clean, s2_clean], ignore_index=True)

# Coerce numeric columns once more (safe)
for c in ["thickness_mm", "width_mm", "weight_kg", "rp02", "rm", "ag", "ai", "quantity"]:
    if c in inventory.columns:
        inventory[c] = pd.to_numeric(inventory[c], errors="coerce")

# Quantity to Int64 (nullable) if all non-null values are whole numbers
q = inventory["quantity"].dropna()
if len(q) == 0 or q.apply(lambda v: float(v).is_integer()).all():
    inventory["quantity"] = inventory["quantity"].astype("Int64")

In [36]:
# Backfill grade from description if still missing
inventory["grade"] = inventory.apply(
    lambda r: r["grade"] if pd.notna(r["grade"]) else extract_grade(r.get("description", "")),
    axis=1
)

# Infer reserved from description if unspecified and 'reserv' substring present
inventory["reserved"] = inventory.apply(
    lambda r: True if (pd.isna(r["reserved"])
                       and isinstance(r.get("description", ""), str)
                       and "reserv" in r["description"].lower()) else r["reserved"],
    axis=1
)

In [37]:
# Tidy description
inventory["description"] = (
    inventory["description"].astype(str)
    .replace({"nan": ""})
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)


In [38]:
# Reorder to canonical schema
inventory = inventory[
    [
        "source", "article_id", "material", "grade", "quality_choice", "finish",
        "thickness_mm", "width_mm", "weight_kg", "quantity",
        "rp02", "rm", "ag", "ai", "reserved", "description"
    ]
].copy()

In [39]:
# Save
# ------------------------
out_path = Path("/content/inventory_dataset.csv")
inventory.to_csv(out_path, index=False)

print(f"Saved: {out_path}  | Rows: {len(inventory)}")

Saved: /content/inventory_dataset.csv  | Rows: 100
